In [1]:
class Room:
    def __init__(self, name, width, height, pixels):
        self.name = name
        self.width = width
        self.height = height
        self.pixels = pixels

    def __str__(self):
        room_str = ""
        for row in self.pixels:
            room_str += "".join(row) + "\n"
        return room_str

# Example usage
a = Room("Kitchen", 6, 6, [
    [' #', ' #', ' #', ' #', ' #', ' #'],
    [' #', ' +', ' +', ' +', ' +', ' #'],
    [' #', ' +', ' A', ' A', ' +', ' #'],
    [' #', ' +', ' A', ' A', ' +', ' #'],
    [' #', ' +', ' +', ' +', ' +', ' #'],
    [' #', ' #', ' #', ' #', ' #', ' #']
])

b = Room("Living Room", 4, 6, [
    [' #', ' #', ' #', ' #'],
    [' #', ' +', ' +', ' #'],
    [' #', ' B', ' B', ' #'],
    [' #', ' B', ' B', ' #'],
    [' #', ' +', ' +', ' #'],
    [' #', ' #', ' #', ' #']
])

c = Room("Bedroom", 6, 4, [
    [' #', ' #', ' #', ' #', ' #', ' #'],
    [' #', ' +', ' C', ' C', ' +', ' #'],
    [' #', ' +', ' C', ' C', ' +', ' #'],
    [' #', ' #', ' #', ' #', ' #', ' #']
])

d = Room("Bathroom", 4, 4, [
    [' #', ' #', ' #', ' #'],
    [' #', ' D', ' D', ' #'],
    [' #', ' D', ' D', ' #'],
    [' #', ' #', ' #', ' #']
])

room_map = {
    0: a,
    1: b,
    2: c,
    3: d
}


In [2]:
class Grid:
    def __init__(self, layout):
        self.layout = layout
        self.height = len(layout)
        self.width = max(len(row) for row in layout)
        self.area = (self.width * self.height) -  sum(row.count('  ') for row in layout)

    def draw_layout(self):
        for row in self.layout:
            print("".join(row))


In [3]:
class RoomGenerator:
    def __init__(self, room_map):
        self.room_map = room_map
        self.layout = []

    def generate_layout(self, phenotype):
        self.layout = []  # Reset layout for each new phenotype
        x, y = 0, 0  # Starting position

        for i in range(0, len(phenotype), 2):
            room_type = phenotype[i]
            direction = phenotype[i + 1]

            room = self.room_map[room_type]

            # Adjust position based on direction
            if direction == 0:  # Right
                x += room.width 
            elif direction == 1:  # Down
                y += room.height
            elif direction == 2:  # Left
                x -= room.width
            elif direction == 3:  # Up
                y -= room.height 

            # Add the room to the layout list
            self.layout.append((x, y, room))

        return self.layout

    def map_layout_to_grid(self):
            # Find the dimensions of the layout
            max_x = max([x + room.width for x, _, room in self.layout])
            max_y = max([y + room.height for _, y, room in self.layout])

            # Initialize the grid
            grid_layout = [['  ' for _ in range(max_x)] for _ in range(max_y)]

            # Map the rooms to the grid
            for x, y, room in self.layout:
                # Check if the indices are within bounds
                if 0 <= y < max_y and 0 <= x < max_x:
                    for row in range(room.height):
                        for col in range(room.width):
                            # Dynamically adjust grid size if necessary
                            if y + row >= len(grid_layout):
                                grid_layout.append(['  ' for _ in range(max_x)])

                            if x + col >= len(grid_layout[y + row]):
                                grid_layout[y + row].append('  ')

                            # Check if the current pixel is a wall in both the room and the existing layout
                            if room.pixels[row][col] == '#' and grid_layout[y + row][x + col] == '#':
                                # Combine walls into a single pixel width
                                grid_layout[y + row][x + col] = ' #'
                            else:
                                # Otherwise, place the room pixel onto the grid
                                grid_layout[y + row][x + col] = room.pixels[row][col]

            # Create a Grid instance
            grid_instance = Grid(grid_layout)

            return grid_instance


In [4]:
room_generator = RoomGenerator(room_map)

pheno_1 = [0, 3, 3, 2, 2, 1, 1, 0, 3, 1, 3, 2, 0, 3, 2, 0, 1, 1]
result_layout_1 = room_generator.generate_layout(pheno_1)
grid_1 = room_generator.map_layout_to_grid()
print("Layout 1:\n")
grid_1.draw_layout()
print(f"\nArea: {grid_1.area}, Width: {grid_1.width}, Height: {grid_1.height}")

pheno_2 = [1, 0, 3, 1, 3, 2, 0, 3, 1, 0, 2, 1, 3, 1, 0, 0, 2, 3]
result_layout_2 = room_generator.generate_layout(pheno_2)
grid_2 = room_generator.map_layout_to_grid()
print("\nLayout 2:\n")
grid_2.draw_layout()
print(f"\nArea: {grid_2.area}, Width: {grid_2.width}, Height: {grid_2.height}")

pheno_3 = [2, 3, 1, 2, 0, 1, 3, 0, 2, 0, 1, 2, 3, 1, 2, 0, 3, 3]
result_layout_3 = room_generator.generate_layout(pheno_3)
grid_3 = room_generator.map_layout_to_grid()
print("\nLayout 3:\n")
grid_3.draw_layout()
print(f"\nArea: {grid_3.area}, Width: {grid_3.width}, Height: {grid_3.height}")

pheno_4 = [3, 2, 0, 3, 1, 0, 2, 1, 2, 2, 1, 1, 0, 3, 2, 1, 1, 2]
result_layout_4 = room_generator.generate_layout(pheno_4)
grid_4 = room_generator.map_layout_to_grid()
print("\nLayout 4:\n")
grid_4.draw_layout()
print(f"\nArea: {grid_4.area}, Width: {grid_4.width}, Height: {grid_4.height}")



Layout 1:

                
                
 # # # # # #    
 # D # + + #    
 # D # B B #    
 # # # B B #    
     # + + #    
     # # # #    

Area: 32, Width: 8, Height: 8

Layout 2:

         # # # #                
         # + + #                
         # # # # # # # # # # # #
         # + C C + # # + C C + #
 # # # # # + C C + # # + C C + #
 # D D # # # # # # # # # # # # #
 # D D # # # # #     # # # # # #
 # # # # # D D #     # + + + + #
         # D D #     # + A A + #
         # # # #     # + A A + #
                     # + + + + #
                     # # # # # #

Area: 124, Width: 16, Height: 12

Layout 3:

                            
                            
 # # # # # # # # # # # #    
 # D # + + # # + # D D #    
 # D # B B # # + # D D #    
 # # # B B # # # # # # #    
     # # # #     # # # # # #
     # D D #     # + C C + #
     # D D #     # + C C + #
     # # # #     # # # # # #

Area: 88, Width: 14, Height: 10

Layout 4:

            
            
       